# Computation with Xarray

- Aggregation: calculation of statistics (e.g. sum) along a dimension of an xarray object can be done by dimension name instead of an integer axis number.
- Arithmetic: arithmetic between xarray objects vectorizes based on dimension names, automatically looping (broadcasting) over each distinct dimension. This eliminates the need to insert dummy dimensions of size one to facilitate broadcasting, a common pattern with NumPy.
- Split-apply-combine: xarray includes N-dimensional grouped operations implementing the split-apply-combine strategy [24].
- Resampling and rolling window operations: Utilizing the efficient resampling methods from pandas and rolling window operations from Bottleneck [15], xarray offers a robust set of resampling and rolling window operations along a single dimension.


### Outline
- Arithmetic
- Aggregation
- Groupby and resample
- Rolling
- Broadcasting

### Tutorial Duriation
10 minutes

In [ ]:
import xarray as xr
ds = xr.tutorial.load_dataset('rasm')
da = ds['Tair']

ds



## Aggregation

Xarray supports many of the aggregations methods that numpy has. A partial list includes: all, any, argmax, argmin, max, mean, median, min, prod, sum, std, var.

Whereas the numpy syntax would require scalar axes, xarray can use dimension names:

In [ ]:
da.mean(dim=('x', 'y'))

## Arithmetic

In [ ]:
# dataarray + scalars
da - 273.15  # (K --> C)

Here we do two computations:
1. Calculate the time-mean
2. Calculate the "anomalies" relative to the time mean

In [ ]:
da_mean = da.mean(dim='time')
da_mean

In [ ]:
# dataarray + dataarray
da - da_mean  

# Notice that this required broadcasting along the time dimension